In [1]:
from collections import Counter
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [2]:
# setup HW
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# check HW availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Num GPUs Available:  0
Num CPUs Available:  1


In [3]:
# load data
dataset_df = pd.read_csv('data/hackprague_txs.csv', sep = ";")
dataset_df.head()

/Users/janmaly/miniconda3/envs/my-alternatives/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,transaction_id,tx_date,client_id,client_gender,client_year_of_birth,amount,merchant_uid,merchant_category,shop_tags,merchant_category_id,shop_uid,shop_type,region,country
0,1,2020-07-02,146167,Female,1980.0,-594,X0noQaM22AHzkPN9erAb74,Professional Services,{Delivery Service},1483998.0,zYMe28Vm9LFEpAN18vaQvG,MERCHANT,Pardubice,CZ
1,2,2020-06-11,40161,Female,1980.0,-986,X0noQaM22AHzkPN9erAb74,Professional Services,{Delivery Service},1483998.0,zYMe28Vm9LFEpAN18vaQvG,MERCHANT,Pardubice,CZ
2,3,2020-08-14,200977,Female,1965.0,-102,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
3,4,2020-06-10,200977,Female,1965.0,-158,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ
4,5,2020-05-27,200977,Female,1965.0,-126,g0DedJv2k5uqz5bYEjdQn8,Food And Drink,NaN,147.0,MkBX1n9BjZFAGoly0Yq3Xy,MERCHANT,Kladno,CZ


In [4]:
# map categories

mapping = [
    {
        'category': 'Bakery',
        'type': 'Groceries',
        'tags': ['Bakery'],
        'places': ['bakery']
     },
    {
        'category': 'Alcohol',
        'type': 'Groceries',
        'tags': ['Alcohol'],
        'places': ['liquor store']
     },
    {
        'category': 'Restaurant',
        'type': 'Food And Drink',
        'tags': ['Restaurant', 'Cuisine', 'Fast Food', 'Bageterie', 'Steak House / Burgers', 'Sushi'],
        'places': ['restaurant']
     },
    {
        'category': 'Party',
        'type': 'Food And Drink',
        'tags': ['Bar', 'Music Club', 'Alcohol', 'Wine', 'Drinking', 'Bear', 'Pub'],
        'places': ['night_club', 'bar', 'pub']
     },
    {
        'category': 'Clothing',
        'type': 'Fashion',
        'tags': ['Fashion Chain - Mix', 'Fashion Accessories', 'Clothing - Other'],
        'places': ['clothing_store']
     },
    {
        'category': 'Pets',
        'type': 'Pets',
        'tags': ['Pet Supplies', 'Pets'],
        'places': ['pet store']
     }      
]

def map_category(x):
    cat: str = x[0]
    tags: str = x[1]

    if pd.isnull(tags):
        return None

    for item in mapping:
        if item['type'] == cat:
            if any(x in tags for x in item['tags']):
                return item['category']
    return None

dataset_filtered_df = dataset_df[dataset_df['merchant_category'].isin(list(map(lambda x: x['type'], mapping))) == True]
dataset_filtered_df['category'] = dataset_filtered_df[['merchant_category','shop_tags']].apply(map_category, axis=1)
dataset_filtered_df = dataset_filtered_df[~dataset_filtered_df['category'].isnull() & ~dataset_filtered_df['merchant_uid'].isnull()]
dataset_filtered_df.head()

<ipython-input-4-da1cc78c523e>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_filtered_df['category'] = dataset_filtered_df[['merchant_category','shop_tags']].apply(map_category, axis=1)


,transaction_id,tx_date,client_id,client_gender,client_year_of_birth,amount,merchant_uid,merchant_category,shop_tags,merchant_category_id,shop_uid,shop_type,region,country,category
2749,2750,2020-06-14,200428,Male,1980.0,-105,beXJ9o0k3NSl9PaoEz4eR3,Food And Drink,"{Bar,Drinking}",147.0,EWAavyAo2GT5LYqkPPoEnj,MERCHANT,Hlavní město Praha,CZ,Party
2750,2751,2020-09-19,169335,Male,1995.0,-89,beXJ9o0k3NSl9PaoEz4eR3,Food And Drink,"{Bar,Drinking}",147.0,EWAavyAo2GT5LYqkPPoEnj,MERCHANT,Hlavní město Praha,CZ,Party
2751,2752,2020-09-04,137596,Male,1985.0,-950,beXJ9o0k3NSl9PaoEz4eR3,Food And Drink,"{Bar,Drinking}",147.0,EWAavyAo2GT5LYqkPPoEnj,MERCHANT,Hlavní město Praha,CZ,Party
2752,2753,2020-08-28,111554,Female,1990.0,-894,beXJ9o0k3NSl9PaoEz4eR3,Food And Drink,"{Bar,Drinking}",147.0,EWAavyAo2GT5LYqkPPoEnj,MERCHANT,Hlavní město Praha,CZ,Party
2753,2754,2020-08-19,40631,Male,1995.0,-644,beXJ9o0k3NSl9PaoEz4eR3,Food And Drink,"{Bar,Drinking}",147.0,EWAavyAo2GT5LYqkPPoEnj,MERCHANT,Hlavní město Praha,CZ,Party


In [5]:
# filter users from Prague
users_in_prague = Counter(dataset_filtered_df[dataset_filtered_df['region'].str.contains("Praha") == True]['client_id'])
users_transactions = Counter(dataset_filtered_df[dataset_filtered_df['client_id'].isin(list(users_in_prague.keys()))]['client_id'])
users_to_keep = [k for k, v in users_in_prague.items() if (v / users_transactions[k]) > 0.8]

# update dataset
dataset_filtered_df = dataset_filtered_df.where(dataset_filtered_df['client_id'].isin(users_to_keep))

In [76]:
# prepare data
visits_df = dataset_filtered_df[~dataset_filtered_df['category'].isnull()
                             & ~dataset_filtered_df['merchant_uid'].isnull()][['client_id', 'merchant_uid']]\
    .drop_duplicates()

In [82]:
# create TF dataset - places + visits
visits = tf.data.Dataset.from_tensor_slices(np.array([[x[0], str(x[1])] for x in visits_df.values])).map(lambda x: {
    "merchant_uid": x[1],
    "client_id": x[0],
})
places = tf.data.Dataset.from_tensor_slices(visits_df['merchant_uid'].unique())

In [83]:
# split dataset
shuffled = visits.shuffle(len(visits_df), reshuffle_each_iteration=False)

train = shuffled.take(int(len(visits_df) * 0.8))
test = shuffled.skip(int(len(visits_df) * 0.8)).take(int(len(visits_df) * 0.2))

In [84]:
# create model
unique_places = visits_df['merchant_uid'].unique().astype(str)
unique_user_ids = visits_df['client_id'].unique().astype(str)

user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, 8)
])

place_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_places, mask_token=None),
  tf.keras.layers.Embedding(len(unique_places) + 1, 8)
])

In [85]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=places.batch(128).map(place_model)
)
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class PlacesModel(tfrs.Model):

  def __init__(self, user_model, place_model):
    super().__init__()
    self.place_model: tf.keras.Model = place_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["client_id"])
    # And pick out the place features and pass them into the movie model,
    # getting embeddings back.
    positive_place_embeddings = self.place_model(features["merchant_uid"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_place_embeddings)

In [92]:
# train model
model = PlacesModel(user_model, place_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(len(visits_df)).batch(128).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
1105/1105 [==============================] - 39s 36ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0537 - factorized_top_k/top_10_categorical_accuracy: 0.0972 - factorized_top_k/top_50_categorical_accuracy: 0.2822 - factorized_top_k/top_100_categorical_accuracy: 0.4018 - loss: 575.2873 - regularization_loss: 0.0000e+00 - total_loss: 575.2873

In [93]:
# evaluate the model
model.evaluate(cached_test, return_dict=True)

9/9 [==============================] - 2s 267ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0044 - factorized_top_k/top_5_categorical_accuracy: 0.0225 - factorized_top_k/top_10_categorical_accuracy: 0.0434 - factorized_top_k/top_50_categorical_accuracy: 0.1596 - factorized_top_k/top_100_categorical_accuracy: 0.2552 - loss: 31034.7840 - regularization_loss: 0.0000e+00 - total_loss: 31034.7840

{'factorized_top_k/top_1_categorical_accuracy': 0.004358038306236267,
 'factorized_top_k/top_5_categorical_accuracy': 0.02246936596930027,
 'factorized_top_k/top_10_categorical_accuracy': 0.04343888908624649,
 'factorized_top_k/top_50_categorical_accuracy': 0.15963438153266907,
 'factorized_top_k/top_100_categorical_accuracy': 0.25519993901252747,
 'loss': 20025.3125,
 'regularization_loss': 0,
 'total_loss': 20025.3125}

In [94]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends places out of the entire movies dataset.
index.index(places.batch(100).map(model.place_model), places)

# Get recommendations.
scores, titles = index(tf.constant([str(users_to_keep[0])]))
print(f"Recommendations for user: {titles[0, :3]}")


Recommendations for user: [b'9LRQyQRlZnUVg2g7rl3kA7' b'jaKZpdXVlcDENq2gq0a0d'
 b'yWblvb4RDRcY7jl3zrJBn8']
